In [1]:
# 1. Read in and clean Toronto neighbourhood table from wikipedia.

In [2]:
import numpy as np #for handling data in a vectorized manner
import pandas as pd #for data analysis
pd.set_option('display.max_columns', None) #ajust columns so all text can be seen
pd.set_option('display.max_rows', None)
import requests #for handling requests
import matplotlib.cm as cm #for accessing colourmaps
import matplotlib.colors as colors #for altering colors
from sklearn.cluster import KMeans #for k-means clustering
import folium #for map rendering
import geocoder #for finding coordinates of sites
import json #for handling json files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url) #read table from page
data = data[0] #convert data to appropriate format
data = data.query("'Not assigned' not in Borough") #remove rows with no borough assigned
data.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [4]:
data = data.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index() 
#combine rows with similar postcodes, grouping together the shared neighbourhoods

#replace unassigned neighbourhood names with borough names
for index, row in data.iterrows():
   if 'Not assigned' in row['Neighbourhood']:
      data.at[index, 'Neighbourhood'] = row['Borough']
        
data.shape


(103, 3)

In [8]:
#2. Add longitudes and latitudes of each neighbourhood to dataframe

In [1]:
latitude = [] #list of latitudes to be converted into column of dataframe
longitude = []

lat_lng_coords = None #initialise variable to none

#obtain coordinates using geocoder.arcgis
for code in data.Postcode:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    

NameError: name 'data' is not defined

In [11]:
#add latitude and longitude columns to data

data['Latitude'] = latitude
data['Longitude'] = longitude

data.to_pickle('toronto_clustering') #store data in pickle

In [3]:
data = pd.read_pickle('toronto_clustering')

In [5]:
#create map of Toronto using latitude and longitude values

import geocoder

g = geocoder.arcgis('Toronto, Ontario') 
lat_lng = g.latlng

map_toronto = folium.Map(location=[lat_lng[0], lat_lng[1]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [1]:
#3. Prepare foursquare credentials and obtain information on venues

CLIENT_ID = '...' # your Foursquare ID
CLIENT_SECRET = '...' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: ...
CLIENT_SECRET:...


In [5]:
#restrict neighbourhoods to those with Toronto in the name of their borough

data = data[data.Borough.str.contains('Toronto', case = False)].reset_index(drop = True) #case refers to alphabet
data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676845,-79.295225
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673
3,M4M,East Toronto,Studio District,43.662766,-79.334830
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085
5,M4P,Central Toronto,Davisville North,43.712815,-79.388526
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
7,M4S,Central Toronto,Davisville,43.703395,-79.385964
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.690655,-79.383561
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686083,-79.402335


In [6]:
#lets get the top 100 venues with in 500 m of each neighbourhood using Foursquare API

LIMIT = 100 #

def getNearbyVenues(names, latitudes, longitudes, radius = 500): #will pass pandas series to this function
    venues_list = []
    for name, lat, lng, in zip(names, latitudes, longitudes):
        print(name)
        #create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
  
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
    
        # extract relevant information for nearby venues
        venues_list.append([(
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])
    
    #convert list into dataframe
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
     
toronto_venues = getNearbyVenues(names=data['Neighbourhood'],
                                 latitudes=data['Latitude'],
                                 longitudes=data['Longitude'])

toronto_venues.head(5)

The Beaches


KeyError: 'groups'

In [18]:
#lets analyze the neighbourhoods. We aim to create a data frame corresponding each neighbourhood
#to its 10 most common venues

In [44]:
#perform one hot encoding using pd.get_dummies command
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="") #prefix refers to new data column name

#add neighbourhood column to toronto_onehot and move it to the first column
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']

#move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [27]:
#next group rows by neighbourhood and take mean of frequency of occurence of each category

In [45]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.0,0.000000,0.020000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.030000,0.000000,0.000000,0.01,0.030000,0.010000,0.00,0.000000,0.000000,0.060000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.01,0.030000,0.000000,0.000000,0.01,0.000000,0.01,0.000000,0.010000,0.000000,0.030000,0.010000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.040000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [46]:
#lets write a function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
#create dataframe displaying top 10 venues for each neighbourhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd'] #for 1'st', 2'nd', and 3'rd' for columns in to be made dataframe
 
#create column names
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
#create new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns = columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Gastropub,Bar,Japanese Restaurant,Burger Joint,Restaurant,Seafood Restaurant,Breakfast Spot
1,Berczy Park,Coffee Shop,Seafood Restaurant,Restaurant,Cocktail Bar,Hotel,Café,Steakhouse,Beer Bar,Cheese Shop,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Furniture / Home Store,Sandwich Place,Restaurant,Bakery,Italian Restaurant,Supermarket,Beer Bar,Poutine Place
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Hotel,Café,Steakhouse,Bar,Restaurant,Pizza Place,American Restaurant,Italian Restaurant,Thai Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Bar,Restaurant,Café,Bakery,Sandwich Place,Speakeasy,Park,Pub


In [ ]:
#4. Cluster the neighbourhoods

In [52]:
clusters = 4 #no, of clusters
toronto_grouped_clusters = toronto_grouped.drop(['Neighbourhood'], 1) #drop neighbourhoods column
kmeans = KMeans(n_clusters= clusters, random_state=0).fit(toronto_grouped_clusters) #run kmeans algorithm
kmeans.labels_ #check cluster labels generated for each row in the dataframe


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 2, 0, 2, 1,
       0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

In [49]:
#create new datafame which includes clusters as well as top 10 venues

In [53]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) #insert column at index 0
toronto_merged = data
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on = 'Neighbourhood')
toronto_merged = toronto_merged.dropna().reset_index() 
#a row of NaNs was added in the process which needs to be dropped
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int') 
#in this process the 'Cluster Labels' integers values were converted to floats, so convert back to nan
toronto_merged.head()

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M4E,East Toronto,The Beaches,43.676845,-79.295225,0,Health Food Store,Other Great Outdoors,Trail,Pub,Coffee Shop,Neighborhood,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120,1,Park,Bus Line,Grocery Store,Discount Store,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
2,2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673,0,Sandwich Place,Gym,Ice Cream Shop,Fish & Chips Shop,Burrito Place,Burger Joint,Liquor Store,Light Rail Station,Food & Drink Shop,Steakhouse
3,3,M4M,East Toronto,Studio District,43.662766,-79.334830,0,Bakery,Diner,Italian Restaurant,Coffee Shop,American Restaurant,Pizza Place,Café,Sushi Restaurant,Arts & Crafts Store,Brewery
4,4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085,1,Bus Line,Swim School,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [54]:
#visualize clusters

g = geocoder.arcgis('Toronto, Ontario') #arcgis api seems to be more reliable than google
lat_lng = g.latlng

map_clusters = folium.Map(location=[lat_lng[0], lat_lng[1]], zoom_start=10)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [55]:
#5. Compare clusters

In [59]:
#cluster 0, defined by mainly restaurents and bars, concentrated by the seafront i.e. the centre of town. Coffee shops
#and cafes are very common

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Health Food Store,Other Great Outdoors,Trail,Pub,Coffee Shop,Neighborhood,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
2,"The Beaches West, India Bazaar",Sandwich Place,Gym,Ice Cream Shop,Fish & Chips Shop,Burrito Place,Burger Joint,Liquor Store,Light Rail Station,Food & Drink Shop,Steakhouse
3,Studio District,Bakery,Diner,Italian Restaurant,Coffee Shop,American Restaurant,Pizza Place,Café,Sushi Restaurant,Arts & Crafts Store,Brewery
7,Davisville,Dessert Shop,Sandwich Place,Café,Pizza Place,Italian Restaurant,Coffee Shop,Gym,Indian Restaurant,Fast Food Restaurant,Seafood Restaurant
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Yoga Studio,Ethiopian Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
11,"Cabbagetown, St. James Town",Café,Coffee Shop,Bakery,Italian Restaurant,Restaurant,Pizza Place,Pet Store,Butcher,Sandwich Place,Chinese Restaurant
12,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Smoke Shop,Gastropub,Fast Food Restaurant,Dance Studio,Men's Store
13,"Harbourfront, Regent Park",Coffee Shop,Gym / Fitness Center,Breakfast Spot,Restaurant,Yoga Studio,Bakery,Spa,Mexican Restaurant,Food Truck,Event Space
14,"Ryerson, Garden District",Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Ramen Restaurant,Tea Room,Restaurant,Fast Food Restaurant,Japanese Restaurant
15,St. James Town,Coffee Shop,Restaurant,Café,Hotel,Bakery,Breakfast Spot,Cocktail Bar,Clothing Store,Cosmetics Shop,Italian Restaurant


In [60]:
#cluster 1, the only areas where bus lines are amongst the most common venues, are far out of town
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"The Danforth West, Riverdale",Park,Bus Line,Grocery Store,Discount Store,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
4,Lawrence Park,Bus Line,Swim School,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [61]:
#cluster 2, areas far out of town but with many shops and exercise areas such as playgrounds, parks, gyms etc.
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Davisville North,Clothing Store,Hotel,Breakfast Spot,Gym,Food & Drink Shop,Park,Deli / Bodega,Ethiopian Restaurant,Flea Market,Fish Market
6,North Toronto West,Playground,Gym Pool,Park,Garden,Yoga Studio,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
8,"Moore Park, Summerhill East",Convenience Store,Gym,Restaurant,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
10,Rosedale,Bank,Park,Playground,Building,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
19,"Harbourfront East, Toronto Islands, Union Station",Harbor / Marina,Music Venue,Café,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
33,"High Park, The Junction South",Convenience Store,Park,Bowling Alley,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [63]:
#cluster 3, areas with not too many amenities but with bus lines not a common venue, could be grouped with cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,"Forest Hill North, Forest Hill West",Park,Yoga Studio,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
